#### Run the following chunk of code to import all of the necessary libraries and packages to run the rest of the script.

In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D, Input, BatchNormalization, Activation
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import Sequence

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os as os
import shutil
import cv2 as cv2

from random import randint
from random import random

#### Run the following chunk of code to move all original tiles and the one-hot encoded truth arrays to a "flow folder". The script will also generate a .csv file, containing file names of images, as well as file names of one-hot encoded truth arrays.

Users will have to adjust the base_Directory variable for their own use. Please note the file layout convention.

In [4]:
base_Directory = ""
image_Names = os.listdir(base_Directory + "Raw_Training_Tiles_Resized/")
mask_Names = os.listdir(base_Directory + "Training_Tiles_Labeled_And_Layered_Resized/")
flow_Directory = base_Directory + "Flow_Folder"
data_Frame_Location = base_Directory + "Excel_Directory/Training_File_Names.csv"

dict = {'X': image_Names, 'Y_True': mask_Names} 
training_Data_Frame = pd.DataFrame(dict)
training_Data_Frame.to_csv(data_Frame_Location)

for file in os.listdir(base_Directory + "Raw_Training_Tiles_Resized/"):
  shutil.copy(base_Directory + "Raw_Training_Tiles_Resized/" + file, base_Directory + "Flow_Folder/" + file)

for file in os.listdir(base_Directory + "Training_Tiles_Labeled_And_Layered_Resized/"):
  shutil.copy(base_Directory + "Training_Tiles_Labeled_And_Layered_Resized/" + file, base_Directory + "Flow_Folder/" + file)

dict = {'X': np.sort(image_Names), 'Y_True': np.sort(mask_Names)} 
training_Data_Frame = pd.DataFrame(dict)
training_Data_Frame.to_csv(data_Frame_Location)



training_Data_Frame = pd.read_csv(data_Frame_Location)

#### Run the following chunk of code to generate the machine learning network architecture.

In [5]:
# ================= Phase 1 Models ====================

def Phase1_Net(img_size, num_classes):
    inputs = Input(shape=img_size + (3,))

    x = Conv2D(64,kernel_size = 3, strides = (1,1),
                            padding = "same")(inputs)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    
    previous_block_concatenate1 = x
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    x = Conv2D(128,kernel_size = 3, strides = (1,1),
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    previous_block_concatenate2 = x

    concate_block_num = 3
    for filters in [256, 512, 512]:
        x = Conv2D(filters,3, strides = (1,1),
                            padding = "same")(x)
        x = BatchNormalization()(x)
        x = Activation(activations.relu)(x)
        x = Conv2D(filters,3, strides = 1,
                         padding = "same")(x)
        x = BatchNormalization()(x)
        x = Activation(activations.relu)(x)
        x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)
        globals()['previous_block_concatenate%s' % concate_block_num] = x
        concate_block_num = concate_block_num + 1
        print(("No errors for filter size:" + str(filters)))



    x = Conv2D(512,3, strides = 1,
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D(pool_size = (2,2),
                                  strides = (2,2))(x)

    x = Conv2D(512,3, strides = 1,
                            padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = Conv2DTranspose(256,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate5], axis =-1)

    x = Conv2DTranspose(256,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate4],axis=-1)

    x = Conv2DTranspose(128,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate3],axis=-1)
    
    x = Conv2DTranspose(64,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = concatenate([x, previous_block_concatenate2],axis=-1)


    x = Conv2DTranspose(32,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    x = Conv2DTranspose(64,2, strides = (2,2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)


    x = concatenate([x, previous_block_concatenate1],axis=-1)

    x = Conv2D(32,3, strides = (1,1),
                            padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = Conv2D(num_classes,3, strides = (1,1),
                            padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    outputs = Conv2D(num_classes,3, strides = (1,1),
                            activation = 'softmax',
                            padding = 'same',
                            name = 'sRBC_classes')(x)
    model = Model(inputs,outputs)

    return model

# ================ Train Phase 2 Model ================

# training the model for a specific amount of epochs 
def Phase1_train_network(model, X_train, y_train, 
                        X_test, y_test, epochs):
    
    train_history = model.fit(X_train, y_train, epochs=epochs, 
                              validation_data=(X_test, y_test),
                              shuffle = True, verbose = 2)
    return model, train_history

#### Run the following chunk of code to create a data generator, which will read tiles into the training method in batches, with augmentation.

In [6]:
class DataGenerator(Sequence):
    def __init__(self, data_Frame, x_Col, y_Col, directory,tile_Namesake, mask_Namesake, subset = None,
                 horizontal_Flips = False, vertical_Flips = False, rotations = False, batch_Size = 32,
                 split = False, training_Ratio = 1, shuffle = False, dim = (128,128), number_Of_Channels = 3,
                 number_Of_Classes = 2, sample_Mean_Zero_Center_Standarardization = True, number_Of_Training_Images = None):
        self.batch_Size = batch_Size
        self.df = data_Frame
        self.x_Col = x_Col
        self.y_Col = y_Col
        self.dim = dim
        self.directory = directory
        self.subset = subset
        self.sample_Mean_Zero_Center_Standarardization = sample_Mean_Zero_Center_Standarardization
        self.number_Of_Classes = number_Of_Classes
        self.number_Of_Channels = number_Of_Channels
        self.shuffle = shuffle
        self.tile_Names = self.df[self.x_Col]
        self.truth_Names = self.df[self.y_Col]
        self.tile_Namesake = tile_Namesake
        self.mask_Namesake = mask_Namesake
        self.horizontal_Flips = horizontal_Flips
        self.vertical_Flips = vertical_Flips
        self.number_Of_Training_Images = number_Of_Training_Images
        self.index_List = np.arange(number_Of_Training_Images) + 1
        self.rotations = rotations
        self.training_Samples = int(training_Ratio*len(self.index_List))
        if split == True:
            self.train_Index_List = self.index_List[:self.training_Samples]
            self.validate_Index_List = self.index_List[self.training_Samples:]
        else:
            self.train_Index_List = self.index_List[:]
            self.validate_Index_List = []
        if self.shuffle == True:
            self.on_Epoch_End()
    def __len__(self):
        return int(len(self.train_Index_List)/self.batch_Size)
    def __getitem__(self, index):
        if self.subset == "Training":
            indexes = self.train_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        elif self.subset == "Validation":
            indexes = self.validate_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        else:
            indexes = self.train_Index_List[index*self.batch_Size:(index*self.batch_Size) + self.batch_Size]
            X, y_True = self.generate_Batch(indexes)
        return X, y_True
    def on_Epoch_End(self):
        if self.shuffle == True:
            np.random.shuffle(self.train_Index_List)
    def generate_Batch(self,indexes):
        X = np.zeros((self.batch_Size, *self.dim, self.number_Of_Channels))
        y_True = np.zeros((self.batch_Size, *self.dim, self.number_Of_Classes))
        for index in range(len(indexes)):
            if self.sample_Mean_Zero_Center_Standarardization == True:
                img = plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3]
                if np.max(img) == int(np.max(img)) and len(str(np.max(img))) == len(str(int(np.max(img)))):
                    img = img.copy()/255.
                if len(np.shape(img)) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                if np.shape(img)[2] == 4:
                    img = img.copy()[:,:,0:3]
                mask = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
                img, mask = self.augment_Image(img,mask)
                X[index,:,:,:] = self.standard_norm(plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3])
                y_True[index,:,:,:] = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
            else:
                img = plt.imread(self.directory + self.tile_Namesake + str(indexes[index]) + ".png")[:,:,0:3]
                if np.max(img) == int(np.max(img)) and len(str(np.max(img))) == len(str(int(np.max(img)))):
                    img = img.copy()/255.
                if len(np.shape(img)) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                if np.shape(img)[2] == 4:
                    img = img.copy()[:,:,0:3]
                mask = np.load(self.directory + self.mask_Namesake + str(indexes[index]) + ".npy")
                img, mask = self.augment_Image(img,mask)
                X[index,:,:,:] = img
                y_True[index,:,:,:] = mask
        return X, y_True
    def standard_norm(self,img):
        height, width, channels = img.shape
        for channel in range(channels):
            img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
        return img
    def augment_Image(self, image, mask):
        if self.rotations == True:
            random_Integer = randint(1,5)
            image = np.rot90(image.copy(),random_Integer)
            mask = np.rot90(mask.copy(),random_Integer)
        
        if self.horizontal_Flips == True:
            random_Float = random()

            if random_Float < 0.5:
                image = np.flip(image.copy(),0)
                mask = np.flip(mask.copy(),0)
        if self.vertical_Flips == True:
            random_Float = random()

            if random_Float < 0.5:
                image = np.flip(image.copy(),1)
                mask = np.flip(mask.copy(),1)
        return image, mask

#### Run the following chunk of code to train the semantic segmantation machine learning architecture.
Note that the number of epochs is intentially made large. Users are not intended to reach the number_Of_Epochs variable. Instead, the learning will be limited by early stopping, given by the patience variable. This variable gives the number of epochs without improvement by the network before training is automatically stopped. Users may need to change the tile_Names_Style and mask_Names_Style variables to match their naming convention.

In [8]:
tile_Names_Style = "/" 
mask_Names_Style = "/"
img_size = (128,128)
num_classes = 2
number_Of_Epochs = 1000
model = Phase1_Net(img_size, num_classes)
#learning hyperparamters for the training optimizer 
model.compile(Adam(learning_rate=0.001),
                 metrics = ['accuracy'],
                 loss = tf.keras.losses.CategoricalCrossentropy())
train_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=True,
                    horizontal_Flips = True,
                    rotations = True,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Training",
                    number_Of_Training_Images = len(image_Names))
validate_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=False,
                    horizontal_Flips = False,
                    rotations = False,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Validation",
                    number_Of_Training_Images = len(image_Names))
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', restore_best_weights=True)
checkpoint = ModelCheckpoint("Phase_One.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
training_History = model.fit(train_Gen,
                             validation_data = validate_Gen,
                                 epochs=number_Of_Epochs, 
                                 verbose = 1, callbacks = [early, checkpoint])

No errors for filter size:256
No errors for filter size:512
No errors for filter size:512
Epoch 1/1000
17/17 [==============================] - ETA: 0s - loss: 0.3485 - accuracy: 0.9407
Epoch 1: val_loss improved from inf to 0.13796, saving model to Phase_One.h5
17/17 [==============================] - 64s 4s/step - loss: 0.3485 - accuracy: 0.9407 - val_loss: 0.1380 - val_accuracy: 0.9908
Epoch 2/1000
17/17 [==============================] - ETA: 0s - loss: 0.2332 - accuracy: 0.9641
Epoch 2: val_loss improved from 0.13796 to 0.08375, saving model to Phase_One.h5
17/17 [==============================] - 61s 4s/step - loss: 0.2332 - accuracy: 0.9641 - val_loss: 0.0838 - val_accuracy: 0.9914
Epoch 3/1000
17/17 [==============================] - ETA: 0s - loss: 0.1746 - accuracy: 0.9646
Epoch 3: val_loss improved from 0.08375 to 0.05160, saving model to Phase_One.h5
17/17 [==============================] - 65s 4s/step - loss: 0.1746 - accuracy: 0.9646 - val_loss: 0.0516 - val_accuracy: 0.9

Epoch 29/1000
17/17 [==============================] - ETA: 0s - loss: 0.0351 - accuracy: 0.9658
Epoch 29: val_loss did not improve from 0.01380
17/17 [==============================] - 65s 4s/step - loss: 0.0351 - accuracy: 0.9658 - val_loss: 0.0140 - val_accuracy: 0.9914
Epoch 30/1000
17/17 [==============================] - ETA: 0s - loss: 0.0346 - accuracy: 0.9658
Epoch 30: val_loss did not improve from 0.01380
17/17 [==============================] - 66s 4s/step - loss: 0.0346 - accuracy: 0.9658 - val_loss: 0.0139 - val_accuracy: 0.9914
Epoch 31/1000
17/17 [==============================] - ETA: 0s - loss: 0.0340 - accuracy: 0.9658
Epoch 31: val_loss improved from 0.01380 to 0.01368, saving model to Phase_One.h5
17/17 [==============================] - 66s 4s/step - loss: 0.0340 - accuracy: 0.9658 - val_loss: 0.0137 - val_accuracy: 0.9914
Epoch 32/1000
17/17 [==============================] - ETA: 0s - loss: 0.0335 - accuracy: 0.9658
Epoch 32: val_loss did not improve from 0.01368

#### Run the following chunk of code to save the trained network architecture and weights.

In [9]:
model.save(base_Directory + "/Trained_Model/Phase_One_Trained_Network.h5")